# 4.0 Feature engineering
Use our existing features to develop new features, convert categorical values to one-hot vectors, and drop others. We will "fit" to the training data and then transform to the test data

In [1]:
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
   
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
DATA_FOLDER = os.path.join('data', 'processed_2')
SAVE_FOLDER = os.path.join('data', 'final')

### Read in our test and training data

In [3]:
zoopla_df_train = pd.read_csv(os.path.join(DATA_FOLDER, 'zoopla_train.csv'), dtype=str)
zoopla_df_test = pd.read_csv(os.path.join(DATA_FOLDER, 'zoopla_test.csv'), dtype=str)

In [4]:
zoopla_df_train.columns

Index(['listing_id', 'parish', 'post_town', 'postcode', 'latitude',
       'longitude', 'num_bedrooms', 'num_bathrooms', 'price',
       'property_type_general', 'CURRENT_ENERGY_RATING_mode',
       'POTENTIAL_ENERGY_RATING_mode', 'TOTAL_FLOOR_AREA_median',
       'NUMBER_HABITABLE_ROOMS_mode', 'CONSTRUCTION_AGE_BAND_mode',
       'Index of Multiple Deprivation Decile', 'Income Decile',
       'Employment Decile', 'Education and Skills Decile',
       'Health and Disability Decile', 'Crime Decile',
       'Barriers to Housing and Services Decile', 'Living Environment Decile',
       'IDACI Decile', 'IDAOPI Decile', 'PROB_4BAND', 'diff_published_date',
       'last_published_year', 'last_published_month', 'first_published_year',
       'first_published_month'],
      dtype='object')

In [5]:
zoopla_df_train.head()

,listing_id,parish,post_town,postcode,latitude,longitude,num_bedrooms,num_bathrooms,price,property_type_general,CURRENT_ENERGY_RATING_mode,POTENTIAL_ENERGY_RATING_mode,TOTAL_FLOOR_AREA_median,NUMBER_HABITABLE_ROOMS_mode,CONSTRUCTION_AGE_BAND_mode,Index of Multiple Deprivation Decile,Income Decile,Employment Decile,Education and Skills Decile,Health and Disability Decile,Crime Decile,Barriers to Housing and Services Decile,Living Environment Decile,IDACI Decile,IDAOPI Decile,PROB_4BAND,diff_published_date,last_published_year,last_published_month,first_published_year,first_published_month
0,62385013,"Hinckley and Bosworth, unparished area",Hinckley,LE10 0,52.5503,-1.386379,4,2,400000.0,Detached house,5.0,3.0,116.0,5.0,3.0,3,3,2,1,5,5,9,9,4,5,None,0,2022,9,2022,9
1,62716145,"Nuneaton and Bedworth, unparished area",Nuneaton,CV10 9,52.525043000000004,-1.5155049999999999,3,1,250000.0,Semi-detached house,4.0,2.0,97.051,4.0,5.0,6,6,6,4,4,7,6,8,5,9,None,1,2022,11,2022,10
2,62579010,"Nuneaton and Bedworth, unparished area",Nuneaton,CV10 0,52.534293999999996,-1.454733,3,1,290000.0,Semi-detached house,4.0,2.0,75.25,4.0,3.0,9,9,8,8,6,7,8,6,10,8,None,2,2023,1,2022,10
3,62485357,"Hinckley and Bosworth, unparished area",Hinckley,LE10 1,52.544140000000006,-1.364725,4,2,550000.0,Detached house,5.0,3.0,179.0,8.0,1.0,5,5,6,3,3,4,10,4,5,5,None,2,2022,12,2022,9
4,62478555,"Nuneaton and Bedworth, unparished area",Nuneaton,CV10 8,52.519397999999995,-1.493818,3,0,235000.0,Semi-detached house,3.0,2.0,71.0,5.0,5.0,4,4,4,3,4,2,9,5,5,5,None,4,2023,2,2022,9


### Drop house prices greater than 1 million pounds and set target as log
Also remove 0 or negative house prices to ensure log does not blow up

In [6]:
zoopla_df_train = zoopla_df_train[(zoopla_df_train['price'].astype(float) <= 1000000) & (zoopla_df_train['price'].astype(float) >= 0)].reset_index()
zoopla_df_test = zoopla_df_test[(zoopla_df_test['price'].astype(float) <= 1000000) & (zoopla_df_test['price'].astype(float) >= 0)].reset_index()

In [7]:
y_train = np.log(zoopla_df_train['price'].astype(float)).rename('LOG_price')
y_test = np.log(zoopla_df_test['price'].astype(float)).rename('LOG_price')

### Separate out features into numeric (with a further separation for numeric vars we want converted to KMeans clusters), ordered discrete and categoric variables

In [8]:
numeric_cols = ['TOTAL_FLOOR_AREA_median', 'diff_published_date']

numeric_kmeans_cols = ['latitude', 'longitude']

discrete_cols = ['num_bedrooms', 'num_bathrooms', 'CURRENT_ENERGY_RATING_mode',
       'POTENTIAL_ENERGY_RATING_mode', 'CONSTRUCTION_AGE_BAND_mode',
       'Index of Multiple Deprivation Decile', 'Income Decile',
       'Employment Decile', 'Education and Skills Decile',
       'Health and Disability Decile', 'Crime Decile',
       'Barriers to Housing and Services Decile', 'Living Environment Decile',
       'IDACI Decile', 'IDAOPI Decile', 'last_published_year', 'last_published_month', 
        'first_published_year', 'first_published_month']

categoric_cols = ['post_town', 'parish', 'postcode', 'PROB_4BAND', 'property_type_general']

### Transform the four types of variables separately in a pipeline
Set up pipelines. For null values, we use the median in the training data for numeric fields and the mode for discrete and categoric fields.

In [9]:
class KMeansCluster(BaseEstimator, TransformerMixin):
    
    """
    Apply KMeans clustering to longitude and latitude. Want cluster centres fit to training
    data and the test data to be transformed based on the fits
    """
    
    def __init__(self, n_clusters=8):
        self.n_clusters = n_clusters        
            
    def fit(self, X, y=None):
        self.kmeans = KMeans(n_clusters=self.n_clusters, n_init=10, random_state=40)
        self.kmeans.fit(X)
        return self
    
    def transform(self, X):
        cluster_centres = self.kmeans.transform(X).argmin(axis=1)
        X = cluster_centres.reshape(cluster_centres.shape[0], 1) # converts 1D array to 2D#
        X = pd.DataFrame(X, columns=['cluster'])
        self.X = X
        return X
        
        # return original co-ordinates and cluster number
        #print(np.c_[X, cluster_centres])
        #return np.c_[X, cluster_centres]
        
    def get_feature_names_out(self, X):
        return self.X.columns.tolist()


In [10]:
numeric_pipeline = Pipeline([
    ('num_imputer', SimpleImputer(strategy='median', add_indicator=False)),
    ('num_std_scaler', StandardScaler()),
])

numeric_kmeans_pipeline = Pipeline([
    ('num_kmeans_imputer', SimpleImputer(strategy='median', add_indicator=False)),
    ('num_kmeans_cluster', KMeansCluster(8)),
    ('num_kmeans_oh_encoder', OneHotEncoder(drop='first'))
])

discrete_pipeline = Pipeline([
    ('dis_imputer', SimpleImputer(strategy='most_frequent', add_indicator=False)),
    ('dis_std_scaler', StandardScaler()),
])

categoric_pipeline = Pipeline([
    ('cat_imputer', SimpleImputer(strategy='most_frequent', add_indicator=False)),
    ('cat_oh_encoder', OneHotEncoder(drop='first'))
])

Set up a column transformer to handle all four types of columns together

In [11]:
full_pipeline = ColumnTransformer([
    ('numeric', numeric_pipeline, numeric_cols),
    ('numeric_kmeans', numeric_kmeans_pipeline, numeric_kmeans_cols),
    ('discrete', discrete_pipeline, discrete_cols),
    ('categoric', categoric_pipeline, categoric_cols),
])

In [12]:
full_pipeline

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('num_imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('num_std_scaler',
                                                  StandardScaler())]),
                                 ['TOTAL_FLOOR_AREA_median',
                                  'diff_published_date']),
                                ('numeric_kmeans',
                                 Pipeline(steps=[('num_kmeans_imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('num_kmeans_cluster',
                                                  KMeansCluster()),
                                                 ('num_kmeans_oh_encoder'...
                                  'Living Environment Decile', 'IDACI Decile',
                                  'IDAOPI Decile', 'last_published_year',
                                  'last_published_month',
                                  'first_published_year',
                                  'first_published_month']),
                                ('categoric',
                                 Pipeline(steps=[('cat_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('cat_oh_encoder',
                                                  OneHotEncoder(drop='first'))]),
                                 ['post_town', 'parish', 'postcode',
                                  'PROB_4BAND', 'property_type_general'])])

Fit to training data and transform it. Then apply those fitted values to the test data.

In [13]:
X_train = full_pipeline.fit_transform(zoopla_df_train)
X_test = full_pipeline.transform(zoopla_df_test)

Check outputs

In [14]:
X_train[200]

array([ 0.56766955, -0.69912374,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        , -0.01075124,
       -1.78748083,  0.47383716, -0.14959827,  1.02406085, -0.78568427,
       -0.85595421, -1.13108125, -0.28235114, -0.68932755, -0.52118875,
        0.16193018, -1.72755914, -0.77737134, -0.99055356,  0.64745946,
       -0.48232303,  1.49039161, -1.19696784,  1.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ])

In [15]:
full_pipeline.transformers_

[('numeric',
  Pipeline(steps=[('num_imputer', SimpleImputer(strategy='median')),
                  ('num_std_scaler', StandardScaler())]),
  ['TOTAL_FLOOR_AREA_median', 'diff_published_date']),
 ('numeric_kmeans',
  Pipeline(steps=[('num_kmeans_imputer', SimpleImputer(strategy='median')),
                  ('num_kmeans_cluster', KMeansCluster()),
                  ('num_kmeans_oh_encoder', OneHotEncoder(drop='first'))]),
  ['latitude', 'longitude']),
 ('discrete',
  Pipeline(steps=[('dis_imputer', SimpleImputer(strategy='most_frequent')),
                  ('dis_std_scaler', StandardScaler())]),
  ['num_bedrooms',
   'num_bathrooms',
   'CURRENT_ENERGY_RATING_mode',
   'POTENTIAL_ENERGY_RATING_mode',
   'CONSTRUCTION_AGE_BAND_mode',
   'Index of Multiple Deprivation Decile',
   'Income Decile',
   'Employment Decile',
   'Education and Skills Decile',
   'Health and Disability Decile',
   'Crime Decile',
   'Barriers to Housing and Services Decile',
   'Living Environment Decile',
   

Set the column names again

In [16]:
feature_names = full_pipeline.get_feature_names_out()

feature_names_new = [name.split('__')[1] for name in feature_names]
print(feature_names_new)

['TOTAL_FLOOR_AREA_median', 'diff_published_date', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7', 'num_bedrooms', 'num_bathrooms', 'CURRENT_ENERGY_RATING_mode', 'POTENTIAL_ENERGY_RATING_mode', 'CONSTRUCTION_AGE_BAND_mode', 'Index of Multiple Deprivation Decile', 'Income Decile', 'Employment Decile', 'Education and Skills Decile', 'Health and Disability Decile', 'Crime Decile', 'Barriers to Housing and Services Decile', 'Living Environment Decile', 'IDACI Decile', 'IDAOPI Decile', 'last_published_year', 'last_published_month', 'first_published_year', 'first_published_month', 'post_town_Nuneaton', 'parish_Hinckley and Bosworth, unparished area', 'parish_Nuneaton and Bedworth, unparished area', 'parish_Other', 'parish_Stoke Golding', 'postcode_CV10 7', 'postcode_CV10 8', 'postcode_CV10 9', 'postcode_CV11 4', 'postcode_CV11 5', 'postcode_CV11 6', 'postcode_CV11 7', 'postcode_CV12 9', 'postcode_CV13 0', 'postcode_CV13 6', 'postcode_CV9 3', 'postco

In [17]:
X_train = pd.DataFrame(X_train, columns=feature_names_new)
y_train = pd.DataFrame(y_train, columns=['LOG_price'])

X_test = pd.DataFrame(X_test, columns=feature_names_new)
y_test = pd.DataFrame(y_test, columns=['LOG_price'])

### Save transformed train and test sets

In [18]:
try:
    os.mkdir(SAVE_FOLDER)
except OSError:
    pass

save_file_train = os.path.join(SAVE_FOLDER, f'zoopla_train.csv')
save_file_test = os.path.join(SAVE_FOLDER, f'zoopla_test.csv')

zoopla_df_train = pd.concat([X_train, y_train], axis=1)
zoopla_df_test = pd.concat([X_test, y_test], axis=1)

zoopla_df_train.to_csv(save_file_train, index=False)
zoopla_df_test.to_csv(save_file_test, index=False)

In [19]:
for colname in zoopla_df_train.columns:
    display(zoopla_df_train[colname].value_counts(dropna=False))

-0.200393    100
 0.138953     13
-0.385490     13
-0.231242     12
-0.447189     12
            ... 
-0.432983      1
 0.345337      1
-0.458763      1
 2.282999      1
 0.773529      1
Name: TOTAL_FLOOR_AREA_median, Length: 207, dtype: int64

-0.699124    259
-0.325847     96
 0.047430     79
 0.420706     56
 0.793983     39
 1.167259     28
 1.913813     18
 1.540536     13
 2.660366      6
 3.033643      3
 3.406919      2
 3.780196      2
 5.273303      1
 4.526749      1
 9.379346      1
 6.019856      1
 2.287089      1
Name: diff_published_date, dtype: int64

0.0    526
1.0     80
Name: cluster_1, dtype: int64

0.0    536
1.0     70
Name: cluster_2, dtype: int64

0.0    480
1.0    126
Name: cluster_3, dtype: int64

...

0.0    602
1.0      4
Name: property_type_general_Maisonette, dtype: int64

0.0    586
1.0     20
Name: property_type_general_Other/Unknown, dtype: int64

0.0    418
1.0    188
Name: property_type_general_Semi-detached house, dtype: int64

0.0    526
1.0     80
Name: property_type_general_Terraced house, dtype: int64

12.429216    28
12.388394    23
12.468437    23
12.611538    20
12.691580    19
             ..
12.671378     1
12.821245     1
12.936034     1
12.060758     1
13.091904     1
Name: LOG_price, Length: 182, dtype: int64

In [20]:
zoopla_df_train

,TOTAL_FLOOR_AREA_median,diff_published_date,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,num_bedrooms,num_bathrooms,CURRENT_ENERGY_RATING_mode,POTENTIAL_ENERGY_RATING_mode,CONSTRUCTION_AGE_BAND_mode,Index of Multiple Deprivation Decile,Income Decile,Employment Decile,Education and Skills Decile,Health and Disability Decile,Crime Decile,Barriers to Housing and Services Decile,Living Environment Decile,IDACI Decile,IDAOPI Decile,last_published_year,last_published_month,first_published_year,first_published_month,post_town_Nuneaton,"parish_Hinckley and Bosworth, unparished area","parish_Nuneaton and Bedworth, unparished area",parish_Other,parish_Stoke Golding,postcode_CV10 7,postcode_CV10 8,postcode_CV10 9,postcode_CV11 4,postcode_CV11 5,postcode_CV11 6,postcode_CV11 7,postcode_CV12 9,postcode_CV13 0,postcode_CV13 6,postcode_CV9 3,postcode_LE10 0,postcode_LE10 1,postcode_LE10 2,postcode_LE10 3,postcode_LE9 7,PROB_4BAND_Medium,PROB_4BAND_None,property_type_general_Detached house,property_type_general_End terrace house,property_type_general_Flat,property_type_general_Maisonette,property_type_general_Other/Unknown,property_type_general_Semi-detached house,property_type_general_Terraced house,LOG_price
0,0.755945,-0.699124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.075124,0.825820,1.617842,1.700535,-0.534404,-1.164162,-1.237357,-1.544492,-1.481684,-0.244458,-0.112069,0.983099,1.030554,-0.777371,-0.589032,-1.544498,1.117375,-0.576376,0.502342,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.899220
1,0.171376,-0.325847,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.010751,-0.480830,0.473837,-0.149598,0.088982,-0.028729,-0.093148,0.109151,-0.282351,-0.689328,0.706170,-0.248654,0.636538,-0.373352,1.017057,-1.544498,1.574432,-0.576376,0.745100,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,12.429216
2,-0.501176,0.047430,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.010751,-0.480830,0.473837,-0.149598,-0.534404,1.106703,1.051061,0.935973,1.316759,0.200412,0.706170,0.572515,-0.151495,1.646747,0.615535,0.647459,-0.710851,-0.576376,0.745100,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,12.577636
3,2.699468,0.047430,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.075124,0.825820,1.617842,1.700535,-1.157791,-0.407207,-0.474551,0.109151,-0.682129,-1.134197,-0.521189,1.393684,-0.939527,-0.373352,-0.589032,-1.544498,1.802960,-0.576376,0.502342,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,13.217674
4,-0.632287,0.793983,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.010751,-1.787481,-0.670168,-0.149598,0.088982,-0.785684,-0.855954,-0.717670,-0.682129,-0.689328,-1.339428,0.983099,-0.545511,-0.373352,-0.589032,0.647459,-0.482323,-0.576376,0.502342,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,12.367341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,0.246926,0.047430,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.010751,0.825820,-1.814173,-1.999732,2.270833,-1.921117,-1.618760,-1.957903,-0.682129,-2.023936,-1.748548,-0.659239,-1.333543,-1.585411,-2.195120,0.647459,-0.710851,-0.576376,0.987859,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.542545
602,0.773529,-0.699124,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.075124,2.132471,-0.670168,-0.149598,1.335754,-0.028729,-0.474551,0.109151,0.117426,-0.244458,0.706170,-1.069823,-0.545511,0.030668,-0.187509,-1.544498,1.574432,-0.576376,0.745100,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.